# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [6]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

In [7]:
# 1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]
print("DataFrame Filtrado:")
print(filtered_df.head())

DataFrame Filtrado:
     Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
189         189  OK31456  California             11009.130490      Yes   
236         236  YJ16163      Oregon             11009.130490      Yes   
419         419  GW43195      Oregon             25807.063000      Yes   
442         442  IP94270     Arizona             13736.132500      Yes   
587         587  FJ28407  California              5619.689084      Yes   

     Coverage             Education Effective To Date EmploymentStatus Gender  \
189   Premium              Bachelor           1/24/11         Employed      F   
236   Premium              Bachelor           1/24/11         Employed      F   
419  Extended               College           2/13/11         Employed      F   
442   Premium                Master           2/13/11         Disabled      F   
587   Premium  High School or Below           1/26/11       Unemployed      M   

     ...  Number of Open Complaints Number of Po

In [8]:
# 2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

response_yes_df = df[df['Response'] == 'Yes']
average_claim = response_yes_df.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean().reset_index()
print("\nPromedio de `Total Claim Amount` por Tipo de Póliza y Género para Respondidos 'Yes':")
print(average_claim)


Promedio de `Total Claim Amount` por Tipo de Póliza y Género para Respondidos 'Yes':
      Policy Type Gender  Total Claim Amount
0  Corporate Auto      F          433.738499
1  Corporate Auto      M          408.582459
2   Personal Auto      F          452.965929
3   Personal Auto      M          457.010178
4    Special Auto      F          453.280164
5    Special Auto      M          429.527942


In [9]:
#3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

customers_by_state = df['State'].value_counts().reset_index()
customers_by_state.columns = ['State', 'Number of Customers']
states_with_more_than_500_customers = customers_by_state[customers_by_state['Number of Customers'] > 500]
print("\nEstados con más de 500 clientes:")
print(states_with_more_than_500_customers)



Estados con más de 500 clientes:
        State  Number of Customers
0  California                 3552
1      Oregon                 2909
2     Arizona                 1937
3      Nevada                  993
4  Washington                  888


In [10]:
# 4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

clv_stats = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median']).reset_index()
print("\nValor de Vida del Cliente por Nivel de Educación y Género:")
print(clv_stats)


Valor de Vida del Cliente por Nivel de Educación y Género:
              Education Gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207
